In [2]:
import pandas as pd
import geopandas as gpd
import csv, sys
import numpy as np

from scipy.spatial import cKDTree
from shapely.geometry import Point

In [3]:
def nearest5HistoryPrice(gdA, gdB):

    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=5)
    gdB_nearest0 = gdB.iloc[idx[:,0]].drop(columns="geometry").reset_index(drop=True)
    gdB_nearest0['history_price0'] = gdB_nearest0['單價元平方公尺']
    gdB_nearest0 = gdB_nearest0['history_price0']
    
    gdB_nearest1 = gdB.iloc[idx[:,1]].drop(columns="geometry").reset_index(drop=True)
    gdB_nearest1['history_price1'] = gdB_nearest1['單價元平方公尺']
    gdB_nearest1 = gdB_nearest1['history_price1']
    
    gdB_nearest2 = gdB.iloc[idx[:,2]].drop(columns="geometry").reset_index(drop=True)
    gdB_nearest2['history_price2'] = gdB_nearest2['單價元平方公尺']
    gdB_nearest2 = gdB_nearest2['history_price2']
    
    gdB_nearest3 = gdB.iloc[idx[:,3]].drop(columns="geometry").reset_index(drop=True)
    gdB_nearest3['history_price3'] = gdB_nearest3['單價元平方公尺']
    gdB_nearest3 = gdB_nearest3['history_price3']
    
    gdB_nearest4 = gdB.iloc[idx[:,4]].drop(columns="geometry").reset_index(drop=True)
    gdB_nearest4['history_price4'] = gdB_nearest4['單價元平方公尺']
    gdB_nearest4 = gdB_nearest4['history_price4']
    
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest0,
            pd.Series(dist[:,0], name='是否有歷史價格'),
            gdB_nearest1,
            pd.Series(dist[:,1], name='dist1'),
            gdB_nearest2,
            pd.Series(dist[:,2], name='dist2'),
            gdB_nearest3,
            pd.Series(dist[:,3], name='dist3'),
            gdB_nearest4,
            pd.Series(dist[:,4], name='dist4')
        ], 
        axis=1)
    gdf.loc[gdf['dist0'] == 0, '是否有歷史價格'] = 0.0001
    gdf.loc[gdf['dist0'] == 0, 'dist1'] = 0.0001
    gdf.loc[gdf['dist0'] == 0, 'dist2'] = 0.0001
    gdf.loc[gdf['dist0'] == 0, 'dist3'] = 0.0001
    gdf.loc[gdf['dist0'] == 0, 'dist4'] = 0.0001
    gdf['history_price_same_object'] = gdf.apply(lambda x: (x['history_price0'] / x['是否有歷史價格'] ** 2 + 
                                                x['history_price1'] / x['dist1'] ** 2 + 
                                                x['history_price2'] / x['dist2'] ** 2 +
                                                x['history_price3'] / x['dist3'] ** 2 +
                                                x['history_price4'] / x['dist2'] ** 4)/
                                     (1 / x['是否有歷史價格'] ** 2 + 
                                     1 / x['dist1'] ** 2 +
                                     1 / x['dist2'] ** 2 +
                                     1 / x['dist3'] ** 2 +
                                     1 / x['dist4'] ** 2), axis = 1)
    gdf.drop(['history_price0', 'history_price1', 'history_price2','history_price3','history_price4'],axis=1,inplace=True)
    gdf.drop(['dist1', 'dist2', 'dist3','dist4'],axis=1,inplace=True)
    return gdf

def nearest5HistoryPrice_by_age(gdA, gdB):

    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=5)
    gdB_nearest0 = gdB.iloc[idx[:,0]].drop(columns="geometry").reset_index(drop=True)
    gdB_nearest0['history_price0'] = gdB_nearest0['單價元平方公尺']
    gdB_nearest0 = gdB_nearest0['history_price0']
    
    gdB_nearest1 = gdB.iloc[idx[:,1]].drop(columns="geometry").reset_index(drop=True)
    gdB_nearest1['history_price1'] = gdB_nearest1['單價元平方公尺']
    gdB_nearest1 = gdB_nearest1['history_price1']
    
    gdB_nearest2 = gdB.iloc[idx[:,2]].drop(columns="geometry").reset_index(drop=True)
    gdB_nearest2['history_price2'] = gdB_nearest2['單價元平方公尺']
    gdB_nearest2 = gdB_nearest2['history_price2']
    
    gdB_nearest3 = gdB.iloc[idx[:,3]].drop(columns="geometry").reset_index(drop=True)
    gdB_nearest3['history_price3'] = gdB_nearest3['單價元平方公尺']
    gdB_nearest3 = gdB_nearest3['history_price3']
    
    gdB_nearest4 = gdB.iloc[idx[:,4]].drop(columns="geometry").reset_index(drop=True)
    gdB_nearest4['history_price4'] = gdB_nearest4['單價元平方公尺']
    gdB_nearest4 = gdB_nearest4['history_price4']
    
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest0,
            pd.Series(dist[:,0], name='dist0'),
            gdB_nearest1,
            pd.Series(dist[:,1], name='dist1'),
            gdB_nearest2,
            pd.Series(dist[:,2], name='dist2'),
            gdB_nearest3,
            pd.Series(dist[:,3], name='dist3'),
            gdB_nearest4,
            pd.Series(dist[:,4], name='dist4')
        ], 
        axis=1)
    gdf.loc[gdf['dist0'] == 0, 'dist0'] = 0.0001
    gdf.loc[gdf['dist0'] == 0, 'dist1'] = 0.0001
    gdf.loc[gdf['dist0'] == 0, 'dist2'] = 0.0001
    gdf.loc[gdf['dist0'] == 0, 'dist3'] = 0.0001
    gdf.loc[gdf['dist0'] == 0, 'dist4'] = 0.0001
    gdf['history_price_same_object'] = gdf.apply(lambda x: (x['history_price0'] / x['dist0'] ** 2 + 
                                                x['history_price1'] / x['dist1'] ** 2 + 
                                                x['history_price2'] / x['dist2'] ** 2 +
                                                x['history_price3'] / x['dist3'] ** 2 +
                                                x['history_price4'] / x['dist2'] ** 4)/
                                     (1 / x['是否有歷史價格'] ** 2 + 
                                     1 / x['dist1'] ** 2 +
                                     1 / x['dist2'] ** 2 +
                                     1 / x['dist3'] ** 2 +
                                     1 / x['dist4'] ** 2), axis = 1)
    gdf.drop(['history_price0', 'history_price1', 'history_price2','history_price3','history_price4'],axis=1,inplace=True)
    gdf.drop(['dist0','dist1', 'dist2', 'dist3','dist4'],axis=1,inplace=True)
    return gdf